# __Long Short Term Memory Decoder__

### __Deep Learning__

#### __Project: Image Captioning with Visual Attention__

In [1]:
import os
os.chdir(os.environ["PYTHONPATH"])

import torch
import matplotlib.pyplot as plt
import seaborn as sns

import scripts.data_loading as dl
import scripts.data_preprocessing as dp
from scripts import model

%matplotlib inline
%load_ext autoreload
%autoreload 2


plt.rcParams["figure.figsize"] = (10, 10)
plt.rcParams["image.cmap"] = "plasma"

In [2]:
coco_train = dl.CocoCaptions(
    dl.TRAINING_DATASET_PATHS[dl.DatasetType.TRAIN],
    dp.VGGNET_PREPROCESSING_PIPELINE,
    dp.TextPipeline(),
)

loading annotations into memory...
Done (t=1.98s)
creating index...
index created!


In [3]:
coco_loader = dl.CocoLoader(coco_train, batch_size=2, num_workers=1)
it = iter(coco_loader)
image_batch, caption_batch = next(it)

In [4]:
encoder = model.VGG19Encoder()
feature_maps, feature_mean = encoder.forward(image_batch)

In [6]:
decoder = model.LSTMDecoder(
    num_embeddings=len(coco_train.target_transform.vocabulary),
    embedding_dim=8,
    encoder_dim=feature_mean.shape[-1],
    decoder_dim=16,
    attention_dim=4
)

In [7]:
embeddings = decoder.word_embedding(caption_batch)

In [9]:
print(f"One-hot encoded caption shape = {caption_batch[0].shape}")
print(f"Embedding shape = {embeddings[0].shape}")

One-hot encoded caption shape = torch.Size([51])
Embedding shape = torch.Size([51, 8])


In [10]:
print(caption_batch[0])
print(embeddings[0])

tensor([10000,    12,    78,    22,    34,   850,    31,     4,     0,   182,
            1,  1421,    65,     2,     0,     8,   283,  1486,     0,    34,
          204,     9,  1584,    47,    34,   924,     1,    34,  1118,     5,
          133,   167,    62,    34,  1186,     4,   121,     4,    36,     2,
          244,     6,    12,  1186,     1,   206,     6,   314, 10002,   139,
        10001])
tensor([[-2.3225e+00, -4.6404e-01,  1.1217e+00, -1.9029e+00, -4.7541e-01,
         -3.0222e+00, -2.6728e-02,  1.6189e-01],
        [ 9.2422e-01,  1.5713e-01,  1.1718e-01,  6.4099e-01,  4.6515e-01,
         -6.5121e-01, -1.0691e+00,  1.4528e+00],
        [ 1.3126e+00,  3.5706e-01, -3.4324e-01, -1.1548e+00, -4.2286e-01,
         -1.0998e+00,  4.1202e-01,  9.4997e-01],
        [ 3.8166e-01,  1.4431e+00, -7.4698e-01,  7.4755e-02, -1.6625e+00,
         -1.0354e+00,  1.2164e+00, -7.2012e-01],
        [ 1.5417e+00,  1.1712e+00, -6.5890e-01, -1.3323e+00, -4.1057e-01,
         -1.5926e+00,  3.947

In [11]:
h = decoder.init_h(feature_mean)
c = decoder.init_c(feature_mean)

print(f"Initial h shape = {h.shape}")
print(f"Initial c shape = {c.shape}")

Initial h shape = torch.Size([2, 16])
Initial c shape = torch.Size([2, 16])


In [12]:
# Initial h, c of LSTM computed by MLP(feature_maps_mean)
print(h[0])
print(c[0])

tensor([-0.0481, -0.0754,  0.0214,  0.0341, -0.0656,  0.2027,  0.1674, -0.2163,
         0.1609, -0.1088,  0.0044,  0.1104,  0.1710,  0.1353,  0.1631,  0.1088],
       grad_fn=<SelectBackward>)
tensor([ 0.0339, -0.0613, -0.3734, -0.3021,  0.0724,  0.1590,  0.1821,  0.1939,
         0.1842, -0.0871, -0.1829, -0.0557, -0.0951,  0.2427, -0.0032, -0.1451],
       grad_fn=<SelectBackward>)


In [13]:
# How to get word embeddings of words at particular index of a caption in batch
index = 50
embeddings[:, index]

tensor([[ 0.5722, -1.2307, -0.2007,  0.2388, -0.8338, -0.7333, -2.0244,  0.2926],
        [ 0.5722, -1.2307, -0.2007,  0.2388, -0.8338, -0.7333, -2.0244,  0.2926]],
       grad_fn=<SelectBackward>)

In [14]:
feature_mean

tensor([[0.1412, 0.1195, 0.1757, 0.1956, 0.2014, 0.1619, 0.1296, 0.1212, 0.1625,
         0.2444, 0.2788, 0.3305, 0.3097, 0.3634, 0.2017, 0.1454, 0.0949, 0.0646,
         0.0738, 0.0580, 0.0685, 0.0834, 0.1497, 0.3198, 0.2564, 0.2706, 0.2274,
         0.2668, 0.4305, 0.2778, 0.3106, 0.2719, 0.2167, 0.0923, 0.0908, 0.1187,
         0.3236, 0.6264, 0.3948, 0.4090, 0.3418, 0.3622, 0.3230, 0.1912, 0.2707,
         0.2215, 0.1806, 0.0852, 0.0931, 0.1090, 0.2908, 0.4428, 0.1921, 0.2183,
         0.2172, 0.3397, 0.3572, 0.2309, 0.2297, 0.1774, 0.2011, 0.1361, 0.1507,
         0.1538, 0.2483, 0.2350, 0.1305, 0.1665, 0.2978, 0.4957, 0.3393, 0.1802,
         0.1848, 0.1314, 0.2549, 0.3474, 0.3141, 0.2434, 0.2089, 0.1747, 0.2000,
         0.2210, 0.3853, 0.5794, 0.2008, 0.1367, 0.1783, 0.1296, 0.2880, 0.4784,
         0.5109, 0.4420, 0.3456, 0.3144, 0.3219, 0.3269, 0.3614, 0.5044, 0.2522,
         0.1661, 0.2265, 0.1435, 0.2713, 0.4259, 0.4878, 0.4517, 0.3450, 0.2899,
         0.2741, 0.2950, 0.3

In [15]:
# How to concatenate context and embedding word
torch.cat([embeddings[:, index], feature_mean], dim=1)

tensor([[ 0.5722, -1.2307, -0.2007,  0.2388, -0.8338, -0.7333, -2.0244,  0.2926,
          0.1412,  0.1195,  0.1757,  0.1956,  0.2014,  0.1619,  0.1296,  0.1212,
          0.1625,  0.2444,  0.2788,  0.3305,  0.3097,  0.3634,  0.2017,  0.1454,
          0.0949,  0.0646,  0.0738,  0.0580,  0.0685,  0.0834,  0.1497,  0.3198,
          0.2564,  0.2706,  0.2274,  0.2668,  0.4305,  0.2778,  0.3106,  0.2719,
          0.2167,  0.0923,  0.0908,  0.1187,  0.3236,  0.6264,  0.3948,  0.4090,
          0.3418,  0.3622,  0.3230,  0.1912,  0.2707,  0.2215,  0.1806,  0.0852,
          0.0931,  0.1090,  0.2908,  0.4428,  0.1921,  0.2183,  0.2172,  0.3397,
          0.3572,  0.2309,  0.2297,  0.1774,  0.2011,  0.1361,  0.1507,  0.1538,
          0.2483,  0.2350,  0.1305,  0.1665,  0.2978,  0.4957,  0.3393,  0.1802,
          0.1848,  0.1314,  0.2549,  0.3474,  0.3141,  0.2434,  0.2089,  0.1747,
          0.2000,  0.2210,  0.3853,  0.5794,  0.2008,  0.1367,  0.1783,  0.1296,
          0.2880,  0.4784,  

In [16]:
#predictions, contexts = decoder.forward(feature_maps, feature_mean, caption_batch)

ValueError: too many values to unpack (expected 2)

In [17]:
caption_batch.shape

torch.Size([2, 51])

Number of predicted words == caption_len - 1

In [ ]:
print(predictions.shape)

In [ ]:
predictions[0, 0, :].shape

In [ ]:
predictions[0, 0, :]

In [ ]:
1 / 10_000 

Random guess prediction probability

In [18]:
#embeddings_t, hidden, context = decoder.forward(feature_maps, feature_mean, caption_batch)

In [22]:
context

tensor([[-0.1818, -0.1165,  0.0341, -0.0915,  0.0902, -0.1428,  0.0529,  0.1196],
        [-0.0950,  0.0188, -0.0773, -0.0995, -0.0248, -0.1069,  0.1902,  0.2213]],
       grad_fn=<AddmmBackward>)

In [21]:
hidden

tensor([[-0.0579, -0.1055, -0.0308,  0.0559, -0.2519,  0.0609,  0.2156, -0.0496],
        [ 0.0955, -0.0668, -0.2021,  0.0172, -0.1793,  0.0961,  0.1042, -0.1226]],
       grad_fn=<AddmmBackward>)

In [20]:
embeddings_t

tensor([[-2.3225, -0.4640,  1.1217, -1.9029, -0.4754, -3.0222, -0.0267,  0.1619],
        [-2.3225, -0.4640,  1.1217, -1.9029, -0.4754, -3.0222, -0.0267,  0.1619]],
       grad_fn=<SelectBackward>)

In [23]:
embeddings_t + hidden + context

tensor([[-2.5622, -0.6861,  1.1251, -1.9385, -0.6371, -3.1042,  0.2417,  0.2318],
        [-2.3220, -0.5121,  0.8423, -1.9852, -0.6795, -3.0330,  0.2677,  0.2606]],
       grad_fn=<AddBackward0>)

In [30]:
preds, contexts = decoder.forward(feature_maps, feature_mean, caption_batch)

In [32]:
preds.shape

torch.Size([50, 2, 10004])

In [33]:
contexts.shape

torch.Size([50, 2, 196])